In [2]:
import pandas as pd
import numpy as np
import pymysql
from collections import Counter

### Database에서 불러오기

In [10]:
dbcon = DBconn()

In [13]:
df_video_cate = dbcon.selectDB('YT_video_category_class')

In [14]:
df_video_lists = dbcon.selectDB('YT_video_lists')

In [15]:
df_channel = dbcon.selectDB('youtuber_info')

### 채널 데이터프레임 생성

In [28]:
channel_cate = df_video_lists[['video_id','channel_id']] #채널 연결 위해

In [30]:
df_video_cate = pd.merge(df_video_cate,channel_cate,on=['video_id'],how='left')

In [144]:
df_results = pd.DataFrame(columns=['Comedy'])

### Comedy 추가

In [145]:
df_comedy = df_video_lists[df_video_lists['genre']=='Comedy'].groupby('channel_id').count()
df_comedy['channel_id'] = df_comedy.index
df_results['channel_id'] = df_comedy['channel_id']
df_results = df_results.reset_index(drop=True)
df_results['Comedy'] = df_comedy.reset_index(drop=True)['ads_yn']

In [152]:
df_gaming = df_video_lists[df_video_lists['genre']=='Gaming'].groupby('channel_id').count()

### Game 추가

In [153]:
df_gaming['channel_id'] = df_gaming.index
df_gaming.reset_index(drop=True,inplace=True)
df_gaming = df_gaming[['channel_id','views']]
df_gaming.columns = ['channel_id','Gaming']

In [157]:
df_results = pd.merge(df_results,df_gaming,on='channel_id',how='outer')

### 음악 추가

In [161]:
df_music = df_video_lists[df_video_lists['genre']=='Music'].groupby('channel_id').count()

In [162]:
df_music['channel_id'] = df_music.index
df_music.reset_index(drop=True,inplace=True)
df_music = df_music[['channel_id','views']]
df_music.columns = ['channel_id','Music']

In [164]:
df_results = pd.merge(df_results,df_music,on='channel_id',how='outer')

In [165]:
df_results

,Comedy,channel_id,Gaming,Music
0,8.0,UC-1cuBAO_YVJt94EeIPDGpg,NaN,NaN
1,29.0,UC-2qZNn9rrlNcsTXNJL5izw,NaN,NaN
2,5.0,UC-3GPiUGCTQkVuxAYR9qpwQ,39.0,NaN
3,22.0,UC-4i5hKIXMOTGO8wVDSAXVw,NaN,NaN
4,4.0,UC-60AlFBYJEoItr-u-tQXbw,NaN,NaN
...,...,...,...,...
13113,NaN,UCzxRke1NegX3wic0TM7BAbg,NaN,8.0
13114,NaN,UCzxXBQnJy9guqVNM6KXi0Ig,NaN,26.0
13115,NaN,UCzxwychp2Jv_scCVwPj2Zaw,NaN,1.0
13116,NaN,UCzz0qdb9rWXe0FkBxkW0nNQ,NaN,7.0


## GCP 분류기에 의해 만들어진 YT_video_category_class 에서 카테고리 모으기

In [232]:
df_main_cate = df_video_cate.groupby('channel_id')['main_category'].apply(lambda x: "%s" % '#'.join(x)).apply(lambda x: Counter(x.split("#")))
df_main_cate = df_main_cate.reset_index()
df_main_cate

,channel_id,main_category
0,UC--8ua5dEkuY26fgWkpkz3Q,"{'Arts & Entertainment/Music & Audio': 24, 'Ar..."
1,UC--GbJkv6N0LLuUWeZDaXvw,{'Arts & Entertainment/Music & Audio': 3}
2,UC--L--_f3e5In69IyzMPLRg,"{'People & Society/Religion & Belief': 54, 'Ar..."
3,UC--LgKcZVgffjsxudoXg5pQ,"{'Computers & Electronics/Software': 2, 'Scien..."
4,UC--McyqJNLmOx2qpdT5SkvQ,{'Games/Computer & Video Games': 2}
...,...,...
47798,UCzzaVRozh8jDugQ7FGLabAQ,"{'Food & Drink/Cooking & Recipes': 4, 'Food & ..."
47799,UCzzsPLVj67qyXrVcGmewSmg,"{'Hobbies & Leisure/Outdoors': 7, 'Sports/Team..."
47800,UCzzt6bFEBcy-CoZeFvTSq0A,"{'Food & Drink/Cooking & Recipes': 3, 'Arts & ..."
47801,UCzzt9YrAmPsya3zBMmS9Nmw,{'Sports/Team Sports': 17}


In [198]:
df_main_cate['main_category'].apply(lambda x: x.most_common(3))

0        [(&, 52), (Arts, 26), (Entertainment/Music, 24)]
1           [(&, 6), (Arts, 3), (Entertainment/Music, 3)]
2        [(&, 128), (People, 54), (Society/Religion, 54)]
3           [(Science/Computer, 9), (Science, 9), (&, 8)]
4               [(Games/Computer, 2), (&, 2), (Video, 2)]
                               ...                       
47798             [(&, 28), (Food, 21), (Drink/Food, 17)]
47799      [(&, 12), (Hobbies, 7), (Leisure/Outdoors, 7)]
47800       [(&, 47), (People, 10), (Society/Family, 10)]
47801                   [(Sports/Team, 17), (Sports, 17)]
47802        [(&, 38), (Finance/Credit, 8), (Lending, 8)]
Name: main_category, Length: 47803, dtype: object

In [233]:
df_sub1_cate = df_video_cate.groupby('channel_id')['sub1_category'].apply(lambda x: "%s" % '#'.join(x)).apply(lambda x: Counter(x.split("#")))
df_sub1_cate = df_sub1_cate.reset_index()
df_sub1_cate

,channel_id,sub1_category
0,UC--8ua5dEkuY26fgWkpkz3Q,{'': 30}
1,UC--GbJkv6N0LLuUWeZDaXvw,"{'': 2, 'Arts & Entertainment/Events & Listing..."
2,UC--L--_f3e5In69IyzMPLRg,"{'': 51, 'Arts & Entertainment/Music & Audio':..."
3,UC--LgKcZVgffjsxudoXg5pQ,"{'': 12, 'Computers & Electronics/Programming'..."
4,UC--McyqJNLmOx2qpdT5SkvQ,{'': 2}
...,...,...
47798,UCzzaVRozh8jDugQ7FGLabAQ,"{'Food & Drink/Cooking & Recipes/Desserts': 1,..."
47799,UCzzsPLVj67qyXrVcGmewSmg,"{'Sports/Extreme Sports': 3, '': 12, 'Hobbies ..."
47800,UCzzt6bFEBcy-CoZeFvTSq0A,"{'': 14, 'People & Society/Family & Relationsh..."
47801,UCzzt9YrAmPsya3zBMmS9Nmw,"{'Games/Computer & Video Games': 9, 'Games/Com..."


In [234]:
df_sub1_cate['sub1_category'].apply(lambda x: x.most_common(3))

0                                                 [(, 30)]
1        [(, 2), (Arts & Entertainment/Events & Listing...
2        [(, 51), (Arts & Entertainment/Music & Audio, ...
3        [(, 12), (Computers & Electronics/Programming,...
4                                                  [(, 2)]
                               ...                        
47798    [(, 10), (Food & Drink/Food/Baked Goods, 9), (...
47799    [(, 12), (Sports/Extreme Sports, 3), (Hobbies ...
47800    [(, 14), (Hobbies & Leisure/Special Occasions/...
47801    [(Games/Computer & Video Games, 9), (Games/Com...
47802    [(, 18), (Finance/Insurance/Health Insurance, ...
Name: sub1_category, Length: 47803, dtype: object

In [235]:
df_sub2_cate = df_video_cate.groupby('channel_id')['sub2_category'].apply(lambda x: "%s" % '#'.join(x)).apply(lambda x: Counter(x.split("#")))
df_sub2_cate = df_sub2_cate.reset_index()
df_sub2_cate

,channel_id,sub2_category
0,UC--8ua5dEkuY26fgWkpkz3Q,{'': 30}
1,UC--GbJkv6N0LLuUWeZDaXvw,"{'': 2, 'Arts & Entertainment/Events & Listing..."
2,UC--L--_f3e5In69IyzMPLRg,{'': 69}
3,UC--LgKcZVgffjsxudoXg5pQ,{'': 17}
4,UC--McyqJNLmOx2qpdT5SkvQ,{'': 2}
...,...,...
47798,UCzzaVRozh8jDugQ7FGLabAQ,"{'Food & Drink/Food': 1, '': 19, 'Food & Drink..."
47799,UCzzsPLVj67qyXrVcGmewSmg,{'': 17}
47800,UCzzt6bFEBcy-CoZeFvTSq0A,"{'': 19, 'Hobbies & Leisure/Special Occasions'..."
47801,UCzzt9YrAmPsya3zBMmS9Nmw,"{'Sports/Team Sports/Soccer': 3, 'Games/Comput..."


In [236]:
df_gcp_channel = pd.DataFrame()
df_gcp_channel['category'] = df_main_cate['main_category'] + df_sub2_cate['sub2_category'] + df_sub1_cate['sub1_category']
df_gcp_channel['channel_id'] =df_sub2_cate['channel_id']
df_gcp_channel

,category,channel_id
0,"{'Arts & Entertainment/Music & Audio': 24, 'Ar...",UC--8ua5dEkuY26fgWkpkz3Q
1,"{'Arts & Entertainment/Music & Audio': 3, '': ...",UC--GbJkv6N0LLuUWeZDaXvw
2,"{'People & Society/Religion & Belief': 59, 'Ar...",UC--L--_f3e5In69IyzMPLRg
3,"{'Computers & Electronics/Software': 2, 'Scien...",UC--LgKcZVgffjsxudoXg5pQ
4,"{'Games/Computer & Video Games': 2, '': 4}",UC--McyqJNLmOx2qpdT5SkvQ
...,...,...
47798,"{'Food & Drink/Cooking & Recipes': 8, 'Food & ...",UCzzaVRozh8jDugQ7FGLabAQ
47799,"{'Hobbies & Leisure/Outdoors': 9, 'Sports/Team...",UCzzsPLVj67qyXrVcGmewSmg
47800,"{'Food & Drink/Cooking & Recipes': 3, 'Arts & ...",UCzzt6bFEBcy-CoZeFvTSq0A
47801,"{'Sports/Team Sports': 17, 'Sports/Team Sports...",UCzzt9YrAmPsya3zBMmS9Nmw


In [237]:
df_gcp_channel['counter'] = df_gcp_channel['category'].apply(lambda x: x.most_common(5))

In [238]:
df_gcp_channel

,category,channel_id,counter
0,"{'Arts & Entertainment/Music & Audio': 24, 'Ar...",UC--8ua5dEkuY26fgWkpkz3Q,"[(, 60), (Arts & Entertainment/Music & Audio, ..."
1,"{'Arts & Entertainment/Music & Audio': 3, '': ...",UC--GbJkv6N0LLuUWeZDaXvw,"[(, 4), (Arts & Entertainment/Music & Audio, 3..."
2,"{'People & Society/Religion & Belief': 59, 'Ar...",UC--L--_f3e5In69IyzMPLRg,"[(, 120), (People & Society/Religion & Belief,..."
3,"{'Computers & Electronics/Software': 2, 'Scien...",UC--LgKcZVgffjsxudoXg5pQ,"[(, 29), (Science/Computer Science, 10), (Comp..."
4,"{'Games/Computer & Video Games': 2, '': 4}",UC--McyqJNLmOx2qpdT5SkvQ,"[(, 4), (Games/Computer & Video Games, 2)]"
...,...,...,...
47798,"{'Food & Drink/Cooking & Recipes': 8, 'Food & ...",UCzzaVRozh8jDugQ7FGLabAQ,"[(, 29), (Food & Drink/Food, 20), (Food & Drin..."
47799,"{'Hobbies & Leisure/Outdoors': 9, 'Sports/Team...",UCzzsPLVj67qyXrVcGmewSmg,"[(, 29), (Hobbies & Leisure/Outdoors, 9), (Spo..."
47800,"{'Food & Drink/Cooking & Recipes': 3, 'Arts & ...",UCzzt6bFEBcy-CoZeFvTSq0A,"[(, 33), (People & Society/Family & Relationsh..."
47801,"{'Sports/Team Sports': 17, 'Sports/Team Sports...",UCzzt9YrAmPsya3zBMmS9Nmw,"[(Sports/Team Sports, 17), (Games/Computer & V..."


### 대분류 엑셀 파일에서 카운트

In [268]:
df_topic_cate = pd.read_excel('topic_cate.xlsx')

In [288]:
df_topic_cate['GCP'] = df_topic_cate['GCP'].apply(lambda x : x[1:]) 
df_topic_cate['Hifen'] = df_topic_cate['Hifen'].str.strip()
df_topic_cate

,GCP,Trans,Hifen
0,Adult,성인,F등급
1,Arts & Entertainment,/아트 & 엔터테인먼트,예술 & 디자인
2,Arts & Entertainment/Celebrities & Entertainme...,/아트 & 엔터테인먼트/연예 뉴스,엔터테인먼트
3,Arts & Entertainment/Comics & Animation,/아트 & 엔터테인먼트/애니메이션,만화 & 애니메이션
4,Arts & Entertainment/Comics & Animation/Anime ...,/아트 & 엔터테인먼트/애니메이션/일본 애니메이션,만화 & 애니메이션
...,...,...,...
615,Travel/Tourist Destinations/Beaches & Islands,/여행/관광지/해변 및 섬,여행 및 숙박
616,Travel/Tourist Destinations/Mountain & Ski Res...,/여행/관광지/산악 및 스키 리조트,여행 및 숙박
617,Travel/Tourist Destinations/Regional Parks & G...,/여행/관광지/지역 공원 & 정원,여행 및 숙박
618,Travel/Tourist Destinations/Theme Parks,/여행/관광지/테마파크,여행 및 숙박


In [290]:
v = df_topic_cate.groupby(['Hifen']).GCP.apply(lambda x: x.tolist())
v

Hifen
F등급               [Adult, Sensitive Subjects, Shopping/Tobacco P...
IT & 전자기기         [Computers & Electronics, Computers & Electron...
TV 프로그램           [Arts & Entertainment/TV & Video, Arts & Enter...
게임                [Games, Games/Computer & Video Games, Games/Co...
경제 & 경영           [Finance, Finance/Accounting & Auditing, Finan...
과학                [Science, Science/Astronomy, Science/Biologica...
교육 & 직업           [Jobs & Education, Jobs & Education/Education,...
동물 (펫)            [Pets & Animals, Pets & Animals/Animal Product...
리빙 & 인테리어         [Home & Garden, Home & Garden/Bed & Bath, Home...
만화 & 애니메이션        [Arts & Entertainment/Comics & Animation, Arts...
보드게임 및 기타 게임      [Games/Arcade & Coin-Op Games, Games/Board Gam...
부동산               [Real Estate, Real Estate/Real Estate Listings...
뷰티 & 메이크업         [Beauty & Fitness, Beauty & Fitness/Beauty Pag...
사회 & 이슈           [Home & Garden/Home Safety & Security, Law & G...
산업 & 공업           [Arts & Entertainment/En

In [295]:
v.keys()

Index(['F등급', 'IT & 전자기기', 'TV 프로그램', '게임', '경제 & 경영', '과학', '교육 & 직업',
       '동물 (펫)', '리빙 & 인테리어', '만화 & 애니메이션', '보드게임 및 기타 게임', '부동산', '뷰티 & 메이크업',
       '사회 & 이슈', '산업 & 공업', '소프트웨어 및 기타기기', '스포츠', '엔터테인먼트', '여행 및 숙박', '영화',
       '예술 & 디자인', '음식 및 레시피', '음악', '의료 & 건강', '장난감 & 선물', '주류 및 음료',
       '주식 & 투자', '차량', '책 & 문학 & 역사', '취미 & 레저 & 공예', '캠핑 & 낚시 & 야외활동',
       '패션 & 의류', '피트니스 & 다이어트'],
      dtype='object', name='Hifen')

In [311]:
from collections import Counter        

In [343]:
def topic_counter(col):
    v = df_topic_cate.groupby(['Hifen']).GCP.apply(lambda x: x.tolist())
    counter = Counter()
    for key,value in col:
        for cate in v.keys():
            if(key in v[cate]):
                counter[cate] += value
    return counter.most_common()

In [344]:
df_gcp_channel['topic_counter_dict'] = df_gcp_channel['counter'].apply(lambda x : topic_counter(x))
df_gcp_channel

,category,channel_id,counter,topic_counter,topic_counter_dict
0,"{'Arts & Entertainment/Music & Audio': 24, 'Ar...",UC--8ua5dEkuY26fgWkpkz3Q,"[(, 60), (Arts & Entertainment/Music & Audio, ...","[(음악, 24), (영화, 2), (스포츠, 2), (게임, 1)]","{'음악': 24, '영화': 2, '스포츠': 2, '게임': 1}"
1,"{'Arts & Entertainment/Music & Audio': 3, '': ...",UC--GbJkv6N0LLuUWeZDaXvw,"[(, 4), (Arts & Entertainment/Music & Audio, 3...","[(음악, 3), (엔터테인먼트, 2)]","{'음악': 3, '엔터테인먼트': 2}"
2,"{'People & Society/Religion & Belief': 59, 'Ar...",UC--L--_f3e5In69IyzMPLRg,"[(, 120), (People & Society/Religion & Belief,...","[(사회 & 이슈, 59), (음악, 21), (엔터테인먼트, 2), (스포츠, 2)]","{'사회 & 이슈': 59, '음악': 21, '엔터테인먼트': 2, '스포츠': 2}"
3,"{'Computers & Electronics/Software': 2, 'Scien...",UC--LgKcZVgffjsxudoXg5pQ,"[(, 29), (Science/Computer Science, 10), (Comp...","[(과학, 10), (소프트웨어 및 기타기기, 6), (취미 & 레저 & 공예, 2)]","{'과학': 10, '소프트웨어 및 기타기기': 6, '취미 & 레저 & 공예': 2}"
4,"{'Games/Computer & Video Games': 2, '': 4}",UC--McyqJNLmOx2qpdT5SkvQ,"[(, 4), (Games/Computer & Video Games, 2)]","[(게임, 2)]",{'게임': 2}
...,...,...,...,...,...
47798,"{'Food & Drink/Cooking & Recipes': 8, 'Food & ...",UCzzaVRozh8jDugQ7FGLabAQ,"[(, 29), (Food & Drink/Food, 20), (Food & Drin...","[(음식 및 레시피, 37), (패션 & 의류, 2)]","{'음식 및 레시피': 37, '패션 & 의류': 2}"
47799,"{'Hobbies & Leisure/Outdoors': 9, 'Sports/Team...",UCzzsPLVj67qyXrVcGmewSmg,"[(, 29), (Hobbies & Leisure/Outdoors, 9), (Spo...","[(캠핑 & 낚시 & 야외활동, 9), (스포츠, 7), (피트니스 & 다이어트, 4)]","{'캠핑 & 낚시 & 야외활동': 9, '스포츠': 7, '피트니스 & 다이어트': 4}"
47800,"{'Food & Drink/Cooking & Recipes': 3, 'Arts & ...",UCzzt6bFEBcy-CoZeFvTSq0A,"[(, 33), (People & Society/Family & Relationsh...","[(사회 & 이슈, 15), (취미 & 레저 & 공예, 15)]","{'사회 & 이슈': 15, '취미 & 레저 & 공예': 15}"
47801,"{'Sports/Team Sports': 17, 'Sports/Team Sports...",UCzzt9YrAmPsya3zBMmS9Nmw,"[(Sports/Team Sports, 17), (Games/Computer & V...","[(게임, 28), (스포츠, 23)]","{'스포츠': 23, '게임': 28}"


In [318]:
df_gcp_channel['topic_counter'] = df_gcp_channel['counter'].apply(lambda x : topic_counter(x))
df_gcp_channel

,category,channel_id,counter,topic_counter
0,"{'Arts & Entertainment/Music & Audio': 24, 'Ar...",UC--8ua5dEkuY26fgWkpkz3Q,"[(, 60), (Arts & Entertainment/Music & Audio, ...","[(음악, 24), (영화, 2), (스포츠, 2), (게임, 1)]"
1,"{'Arts & Entertainment/Music & Audio': 3, '': ...",UC--GbJkv6N0LLuUWeZDaXvw,"[(, 4), (Arts & Entertainment/Music & Audio, 3...","[(음악, 3), (엔터테인먼트, 2)]"
2,"{'People & Society/Religion & Belief': 59, 'Ar...",UC--L--_f3e5In69IyzMPLRg,"[(, 120), (People & Society/Religion & Belief,...","[(사회 & 이슈, 59), (음악, 21), (엔터테인먼트, 2), (스포츠, 2)]"
3,"{'Computers & Electronics/Software': 2, 'Scien...",UC--LgKcZVgffjsxudoXg5pQ,"[(, 29), (Science/Computer Science, 10), (Comp...","[(과학, 10), (소프트웨어 및 기타기기, 6), (취미 & 레저 & 공예, 2)]"
4,"{'Games/Computer & Video Games': 2, '': 4}",UC--McyqJNLmOx2qpdT5SkvQ,"[(, 4), (Games/Computer & Video Games, 2)]","[(게임, 2)]"
...,...,...,...,...
47798,"{'Food & Drink/Cooking & Recipes': 8, 'Food & ...",UCzzaVRozh8jDugQ7FGLabAQ,"[(, 29), (Food & Drink/Food, 20), (Food & Drin...","[(음식 및 레시피, 37), (패션 & 의류, 2)]"
47799,"{'Hobbies & Leisure/Outdoors': 9, 'Sports/Team...",UCzzsPLVj67qyXrVcGmewSmg,"[(, 29), (Hobbies & Leisure/Outdoors, 9), (Spo...","[(캠핑 & 낚시 & 야외활동, 9), (스포츠, 7), (피트니스 & 다이어트, 4)]"
47800,"{'Food & Drink/Cooking & Recipes': 3, 'Arts & ...",UCzzt6bFEBcy-CoZeFvTSq0A,"[(, 33), (People & Society/Family & Relationsh...","[(사회 & 이슈, 15), (취미 & 레저 & 공예, 15)]"
47801,"{'Sports/Team Sports': 17, 'Sports/Team Sports...",UCzzt9YrAmPsya3zBMmS9Nmw,"[(Sports/Team Sports, 17), (Games/Computer & V...","[(게임, 28), (스포츠, 23)]"


In [351]:
df_channel_cate = df_gcp_channel['topic_counter_dict'].apply(pd.Series)

In [359]:
df_gcp_channel2 = pd.concat([df_gcp_channel[['channel_id']],df_channel_cate],axis=1)

In [360]:
df_gcp_channel2

,channel_id,음악,영화,스포츠,게임,엔터테인먼트,사회 & 이슈,과학,소프트웨어 및 기타기기,취미 & 레저 & 공예,...,캠핑 & 낚시 & 야외활동,경제 & 경영,주식 & 투자,보드게임 및 기타 게임,부동산,산업 & 공업,리빙 & 인테리어,차량,책 & 문학 & 역사,F등급
0,UC--8ua5dEkuY26fgWkpkz3Q,24.0,2.0,2.0,1.0,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,UC--GbJkv6N0LLuUWeZDaXvw,3.0,NaN,NaN,NaN,2.0,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,UC--L--_f3e5In69IyzMPLRg,21.0,NaN,2.0,NaN,2.0,59.0,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,UC--LgKcZVgffjsxudoXg5pQ,NaN,NaN,NaN,NaN,NaN,NaN,10.0,6.0,2.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,UC--McyqJNLmOx2qpdT5SkvQ,NaN,NaN,NaN,2.0,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
47798,UCzzaVRozh8jDugQ7FGLabAQ,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
47799,UCzzsPLVj67qyXrVcGmewSmg,NaN,NaN,7.0,NaN,NaN,NaN,NaN,NaN,NaN,...,9.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
47800,UCzzt6bFEBcy-CoZeFvTSq0A,NaN,NaN,NaN,NaN,NaN,15.0,NaN,NaN,15.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
47801,UCzzt9YrAmPsya3zBMmS9Nmw,NaN,NaN,23.0,28.0,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [361]:
df_results

,Comedy,channel_id,Gaming,Music
0,8.0,UC-1cuBAO_YVJt94EeIPDGpg,NaN,NaN
1,29.0,UC-2qZNn9rrlNcsTXNJL5izw,NaN,NaN
2,5.0,UC-3GPiUGCTQkVuxAYR9qpwQ,39.0,NaN
3,22.0,UC-4i5hKIXMOTGO8wVDSAXVw,NaN,NaN
4,4.0,UC-60AlFBYJEoItr-u-tQXbw,NaN,NaN
...,...,...,...,...
13113,NaN,UCzxRke1NegX3wic0TM7BAbg,NaN,8.0
13114,NaN,UCzxXBQnJy9guqVNM6KXi0Ig,NaN,26.0
13115,NaN,UCzxwychp2Jv_scCVwPj2Zaw,NaN,1.0
13116,NaN,UCzz0qdb9rWXe0FkBxkW0nNQ,NaN,7.0


In [398]:
df_gcp_channel_final = pd.merge(df_gcp_channel2,df_results,how='outer',on='channel_id')

In [403]:
df_gcp_channel_final = df_gcp_channel_final.fillna(0)

In [404]:
df_gcp_channel_final['음악'] = df_gcp_channel_final['음악_x'] + df_gcp_channel_final['음악_y']

In [406]:
df_gcp_channel_final['게임'] = df_gcp_channel_final['게임_x'] + df_gcp_channel_final['게임_y']

In [408]:
df_gcp_channel_final.drop(['게임_x','게임_y','음악_x','음악_y'],1,inplace=True)

<ipython-input-408-c6d3c1f7eecb>:1: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only
  df_gcp_channel_final.drop(['게임_x','게임_y','음악_x','음악_y'],1,inplace=True)


In [409]:
df_gcp_channel_final

,영화,스포츠,엔터테인먼트,사회 & 이슈,과학,소프트웨어 및 기타기기,취미 & 레저 & 공예,교육 & 직업,의료 & 건강,TV 프로그램,...,보드게임 및 기타 게임,부동산,산업 & 공업,리빙 & 인테리어,차량,책 & 문학 & 역사,F등급,코미디,음악,게임
channel_id,,,,,,,,,,,,,,,,,,,,,
UC--8ua5dEkuY26fgWkpkz3Q,2.0,2.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,24.0,1.0
UC--GbJkv6N0LLuUWeZDaXvw,0.0,0.0,2.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,32.0,0.0
UC--L--_f3e5In69IyzMPLRg,0.0,2.0,2.0,59.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,21.0,0.0
UC--LgKcZVgffjsxudoXg5pQ,0.0,0.0,0.0,0.0,10.0,6.0,2.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
UC--McyqJNLmOx2qpdT5SkvQ,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,28.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
UCyLZwTT87Gp4pAGq_aysSVg,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.0,0.0
UCyYqXmPUO4InZT-E62F7hVg,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,30.0,0.0
UCynBQhiDx0WTsFDiZsQgG0Q,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.0,0.0


In [413]:
df_gcp_channel_final = df_gcp_channel_final.reset_index()

In [414]:
df_gcp_channel_final

,channel_id,영화,스포츠,엔터테인먼트,사회 & 이슈,과학,소프트웨어 및 기타기기,취미 & 레저 & 공예,교육 & 직업,의료 & 건강,...,보드게임 및 기타 게임,부동산,산업 & 공업,리빙 & 인테리어,차량,책 & 문학 & 역사,F등급,코미디,음악,게임
0,UC--8ua5dEkuY26fgWkpkz3Q,2.0,2.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,24.0,1.0
1,UC--GbJkv6N0LLuUWeZDaXvw,0.0,0.0,2.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,32.0,0.0
2,UC--L--_f3e5In69IyzMPLRg,0.0,2.0,2.0,59.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,21.0,0.0
3,UC--LgKcZVgffjsxudoXg5pQ,0.0,0.0,0.0,0.0,10.0,6.0,2.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,UC--McyqJNLmOx2qpdT5SkvQ,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,28.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
48244,UCyLZwTT87Gp4pAGq_aysSVg,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.0,0.0
48245,UCyYqXmPUO4InZT-E62F7hVg,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,30.0,0.0
48246,UCynBQhiDx0WTsFDiZsQgG0Q,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.0,0.0
48247,UCzJwxB-3p8SjL7O5gKdkBow,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,25.0,0.0


In [418]:
df_gcp_channel_final = df_gcp_channel_final.replace(0, np.NaN)

In [383]:
total_columns = v.keys().tolist() + ['코미디']

In [419]:
third_highest_value_index = df_gcp_channel_final[total_columns].apply(lambda s: s.nlargest(3).index.tolist(), axis=1)
df_rank = pd.DataFrame(third_highest_value_index.tolist(), columns=['first_cate','second_cate','third_cate'])

In [420]:
df_rank

,first_cate,second_cate,third_cate
0,음악,스포츠,영화
1,음악,엔터테인먼트,None
2,사회 & 이슈,음악,스포츠
3,과학,소프트웨어 및 기타기기,취미 & 레저 & 공예
4,게임,None,None
...,...,...,...
48244,음악,None,None
48245,음악,None,None
48246,음악,None,None
48247,음악,None,None


In [421]:
df_gcp_channel_final['first_cate'] = df_rank['first_cate'].fillna('')
df_gcp_channel_final['second_cate'] = df_rank['second_cate'].fillna('')
df_gcp_channel_final['third_cate'] = df_rank['third_cate'].fillna('')

In [423]:
df_gcp_channel_final

,channel_id,영화,스포츠,엔터테인먼트,사회 & 이슈,과학,소프트웨어 및 기타기기,취미 & 레저 & 공예,교육 & 직업,의료 & 건강,...,리빙 & 인테리어,차량,책 & 문학 & 역사,F등급,코미디,음악,게임,first_cate,second_cate,third_cate
0,UC--8ua5dEkuY26fgWkpkz3Q,2.0,2.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,24.0,1.0,음악,스포츠,영화
1,UC--GbJkv6N0LLuUWeZDaXvw,NaN,NaN,2.0,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,32.0,NaN,음악,엔터테인먼트,
2,UC--L--_f3e5In69IyzMPLRg,NaN,2.0,2.0,59.0,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,21.0,NaN,사회 & 이슈,음악,스포츠
3,UC--LgKcZVgffjsxudoXg5pQ,NaN,NaN,NaN,NaN,10.0,6.0,2.0,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,과학,소프트웨어 및 기타기기,취미 & 레저 & 공예
4,UC--McyqJNLmOx2qpdT5SkvQ,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,28.0,게임,,
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
48244,UCyLZwTT87Gp4pAGq_aysSVg,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,2.0,NaN,음악,,
48245,UCyYqXmPUO4InZT-E62F7hVg,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,30.0,NaN,음악,,
48246,UCynBQhiDx0WTsFDiZsQgG0Q,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,2.0,NaN,음악,,
48247,UCzJwxB-3p8SjL7O5gKdkBow,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,25.0,NaN,음악,,


In [424]:
df_gcp_channel_final.columns

Index(['channel_id', '영화', '스포츠', '엔터테인먼트', '사회 & 이슈', '과학', '소프트웨어 및 기타기기',
       '취미 & 레저 & 공예', '교육 & 직업', '의료 & 건강', 'TV 프로그램', '음식 및 레시피',
       'IT & 전자기기', '뷰티 & 메이크업', '패션 & 의류', '여행 및 숙박', '동물 (펫)', '예술 & 디자인',
       '만화 & 애니메이션', '장난감 & 선물', '주류 및 음료', '피트니스 & 다이어트', '캠핑 & 낚시 & 야외활동',
       '경제 & 경영', '주식 & 투자', '보드게임 및 기타 게임', '부동산', '산업 & 공업', '리빙 & 인테리어',
       '차량', '책 & 문학 & 역사', 'F등급', '코미디', '음악', '게임', 'first_cate',
       'second_cate', 'third_cate'],
      dtype='object')

In [429]:
eng_col = ['channel_id', 'movie', 'sports', 'entertainment', 'social_issues', 'science', 'software',\
       'hobby_leisure', 'education_job', 'medical_health', 'tv_program', 'food_recipe',\
       'IT_electronics', 'beauty_makeup', 'fashion_cloth', 'trip_accomodation', 'animal_pet', 'art_design',\
       'toon_animation', 'toy_gift', 'beverage', 'fitness_diet', 'camping_outdoor',\
       'economy', 'stock_investment', 'board_game', 'realestate', 'industry', 'living_interior',\
       'car', 'book_history', 'F_level', 'comedy', 'music', 'game', 'first_cate',\
       'second_cate', 'third_cate']

In [430]:
eng_col

['channel_id',
 'movie',
 'sports',
 'entertainment',
 'social_issues',
 'science',
 'software',
 'hobby_leisure',
 'education_job',
 'medical_health',
 'tv_program',
 'food_recipe',
 'IT_electronics',
 'beauty_makeup',
 'fashion_cloth',
 'trip_accomodation',
 'animal_pet',
 'art_design',
 'toon_animation',
 'toy_gift',
 'beverage',
 'fitness_diet',
 'camping_outdoor',
 'economy',
 'stock_investment',
 'board_game',
 'realestate',
 'industry',
 'living_interior',
 'car',
 'book_history',
 'F_level',
 'comedy',
 'music',
 'game',
 'first_cate',
 'second_cate',
 'third_cate']

In [422]:
df_gcp_channel_final.to_csv('final.csv',encoding='utf-8-sig',index=False)

### DataBase에 입력

In [433]:
insert_df = df_gcp_channel_final
insert_df.columns = eng_col

In [434]:
insert_df

,channel_id,movie,sports,entertainment,social_issues,science,software,hobby_leisure,education_job,medical_health,...,living_interior,car,book_history,F_level,comedy,music,game,first_cate,second_cate,third_cate
0,UC--8ua5dEkuY26fgWkpkz3Q,2.0,2.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,24.0,1.0,음악,스포츠,영화
1,UC--GbJkv6N0LLuUWeZDaXvw,NaN,NaN,2.0,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,32.0,NaN,음악,엔터테인먼트,
2,UC--L--_f3e5In69IyzMPLRg,NaN,2.0,2.0,59.0,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,21.0,NaN,사회 & 이슈,음악,스포츠
3,UC--LgKcZVgffjsxudoXg5pQ,NaN,NaN,NaN,NaN,10.0,6.0,2.0,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,과학,소프트웨어 및 기타기기,취미 & 레저 & 공예
4,UC--McyqJNLmOx2qpdT5SkvQ,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,28.0,게임,,
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
48244,UCyLZwTT87Gp4pAGq_aysSVg,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,2.0,NaN,음악,,
48245,UCyYqXmPUO4InZT-E62F7hVg,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,30.0,NaN,음악,,
48246,UCynBQhiDx0WTsFDiZsQgG0Q,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,2.0,NaN,음악,,
48247,UCzJwxB-3p8SjL7O5gKdkBow,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,25.0,NaN,음악,,


In [435]:
from sqlalchemy import create_engine
from sqlalchemy.ext.compiler import compiles
from sqlalchemy.sql.expression import Insert
from sqlalchemy import create_engine
pymysql.install_as_MySQLdb()


In [436]:
engine = create_engine("mysql+mysqldb://"+user+":"+str(passwd)+"@"+host+":"+str(port)+"/"+db, encoding='utf-8')
insert_df.to_sql(name='YT_channel_cate', con=engine, if_exists = 'append', index=False)   